# GOTM at Ocean station Papa

This notebook runs [GOTM](https://gotm.net/) with initial conditions and surface forcing at the [ocean station Papa](https://www.pmel.noaa.gov/ocs/Papa), using computed fluxes.

In [ ]:
import sys
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *

## Create a model
Create a model with environment file `gotmtool/.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [ ]:
m = Model(name='OSPapa_flux', environ=os.path.join(os.pardir, 'gotmtool', '.gotm_env.yaml'))

Take a look at what are defined in the environment file.

In [ ]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

## Build the model

In [ ]:
%%time
m.build()

## Configuration
Initialize the GOTM configuration

In [ ]:
cfg = m.init_config()

Update the configuration

In [ ]:
# setup
title = 'OSPapa (flux)'
nlev = 150
cfg['title'] = title
cfg['location']['name'] = 'Papa'
cfg['location']['latitude'] = 50.1
cfg['location']['longitude'] = -144.9
cfg['location']['depth'] = 150.0
cfg['time']['start'] = '2012-03-21 00:00:00'
cfg['time']['stop']  = '2013-03-21 00:00:00'
cfg['time']['dt']    = 3600.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = os.path.join(os.getcwd(), 'data')
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = os.path.join(datadir, 't_prof.dat')
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = os.path.join(datadir, 's_prof.dat')
cfg['surface']['fluxes']['method'] = 'off'
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['heat']['file'] = os.path.join(datadir, 'heat_flux.dat')
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['file'] = os.path.join(datadir, 'momentum_flux.dat')
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['file'] = os.path.join(datadir, 'momentum_flux.dat')
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['u10']['method'] = 'file'
cfg['surface']['u10']['file'] = os.path.join(datadir, 'u10.dat')
cfg['surface']['u10']['column'] = 1
cfg['surface']['v10']['method'] = 'file'
cfg['surface']['v10']['file'] = os.path.join(datadir, 'u10.dat')
cfg['surface']['v10']['column'] = 2
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = os.path.join(datadir, 'swr.dat')
cfg['surface']['precip']['method'] = 'file'
cfg['surface']['precip']['file'] = os.path.join(datadir, 'pme.dat')
cfg['surface']['calc_evaporation'] = False
cfg['surface']['sst']['method'] = 'file'
cfg['surface']['sst']['file'] = os.path.join(datadir, 'sst.dat')
cfg['surface']['sss']['method'] = 'file'
cfg['surface']['sss']['file'] = os.path.join(datadir, 'sss.dat')

cfg['waves']['stokes_drift']['us0']['method'] = 'file'
cfg['waves']['stokes_drift']['us0']['file'] = os.path.join(datadir, 'us_surface.dat')
cfg['waves']['stokes_drift']['us0']['column'] = 1
cfg['waves']['stokes_drift']['vs0']['method'] = 'file'
cfg['waves']['stokes_drift']['vs0']['file'] = os.path.join(datadir, 'us_surface.dat')
cfg['waves']['stokes_drift']['vs0']['column'] = 2
cfg['waves']['stokes_drift']['us']['method'] = 'file'
cfg['waves']['stokes_drift']['us']['file'] = os.path.join(datadir, 'us_prof.dat')
cfg['waves']['stokes_drift']['us']['column'] = 1
cfg['waves']['stokes_drift']['vs']['method'] = 'file'
cfg['waves']['stokes_drift']['vs']['file'] = os.path.join(datadir, 'us_prof.dat')
cfg['waves']['stokes_drift']['vs']['column'] = 2

# water type (Jerlov 1B)
cfg['light_extinction']['method'] = 'jerlov-ib'


Now run the case with [CVMix](http://cvmix.github.io).
- KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
- StokesMOST ([Large et al., 2021](https://doi.org/10.1175/JPO-D-20-0308.1))Run the case with KPP in [CVMix](http://cvmix.github.io).

In [ ]:
cfgs = []
labels = []

In [ ]:
cfg['turbulence']['turb_method'] = 'cvmix'
cfg['cvmix']['surface_layer']['langmuir_method'] = 'lf17'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')
cfg['cvmix']['surface_layer']['langmuir_method'] = 'none'
cfg['cvmix']['surface_layer']['use_Stokes_MOST'] = True
cfgs.append(copy.deepcopy(cfg))
labels.append('StokesMOST')

## Run the model

In [ ]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

## Results

Load the data into an `xarray.Dataset`.

In [ ]:
data1 = sims[0].load_data()
data2 = sims[1].load_data()

In [ ]:
fig, axarr = plt.subplots(2, 1, sharex='col')
fig.set_size_inches([8,6])
data1.temp.plot(ax=axarr[0])
data2.temp.plot(ax=axarr[1])
for i, ax in enumerate(axarr):
    ax.set_title('')
    ax.set_xlabel('')
    ax.text(0.03, 0.05, labels[i], transform=ax.transAxes, color='w', fontsize=11, va='bottom')